<a href="https://colab.research.google.com/github/chi-hun/dacon/blob/main/%EC%87%BC%ED%95%91%EB%AA%B0_%EB%A6%AC%EB%B7%B0_%ED%8F%89%EC%A0%90_%EB%B6%84%EB%A5%98_%EA%B2%BD%EC%A7%84%EB%8C%80%ED%9A%8C.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

쇼핑몰 리뷰 평점 분류 경진대회 https://dacon.io/competitions/official/235938/overview/description

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


# module

In [ ]:
!pip install konlpy

In [ ]:
!pip install git+https://github.com/ssut/py-hanspell.git

In [ ]:
!pip install catboost

In [25]:
import pandas as pd
import numpy as np
from konlpy.tag import Okt
from hanspell import spell_checker
import re
import tqdm
import sklearn.feature_extraction
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.metrics import accuracy_score
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier, VotingClassifier
from xgboost import XGBRFClassifier
from lightgbm import LGBMClassifier
from catboost import CatBoostClassifier
import tensorflow as tf

# EDA

In [5]:
train = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/데이콘/쇼핑몰 리뷰 평점 분류 경진대회/train.csv')
test = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/데이콘/쇼핑몰 리뷰 평점 분류 경진대회/test.csv')

In [ ]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 25000 entries, 0 to 24999
Data columns (total 3 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   id       25000 non-null  int64 
 1   reviews  25000 non-null  object
 2   target   25000 non-null  int64 
dtypes: int64(2), object(1)
memory usage: 586.1+ KB


In [ ]:
train.head()

,id,reviews,target
0,0,조아요 처음구입 싸게햇어요,2
1,1,생각보다 잘 안돼요 매지 바른지 하루밖에 안됐는데ㅠㅠ 25천원가량 주고 사기 너무 ...,1
2,2,디자인은괜찮은데 상품이 금이가서 교환했는데 두번째받은상품도 까져있고 안쪽에 금이가져...,2
3,3,기전에 이 제품말고 이마트 트레이더스에서만 팔던 프리미엄 제품을 사용했었습니다. 샘...,2
4,4,튼튼하고 손목을 잘 받쳐주네요~,5


In [ ]:
train[train['target'] == 5].head()

,id,reviews,target
4,4,튼튼하고 손목을 잘 받쳐주네요~,5
6,6,타이밍이 좋았는지 배송도 빠르고 맛있었어요! 크기가 두가지가 있길래 큰걸 사봤는데 ...,5
8,8,재구매 평소 사용하던 제품이에요. 한번에 쟁여두려 여러개 삽니다 .,5
10,10,재구매 항상 여기서만 주문해서 잘 쓰고있어요 후딱쓰기좋아요 크기도 좋구,5
12,12,재구매 좋은제품 저렴하게 잘 구매했습니다.,5


In [ ]:
train[train['target'] == 1].head()

,id,reviews,target
1,1,생각보다 잘 안돼요 매지 바른지 하루밖에 안됐는데ㅠㅠ 25천원가량 주고 사기 너무 ...,1
19,19,완전 골아서 왔어요 집에서 썩은내가 진동해서 베란다문 열고 그래도 내장이랑 분리 해...,1
40,40,이상한 모양이 찍혀 나와요 제 프린터의 문제인줄 알았는데 다른 사람도 똑같은 모양이...,1
41,41,양이 너무 적어요 밖에서 사먹는게 더 쌀듯,1
42,42,양념통5개 오일병5개 주문했는데 양념통 뚜껑하나가 스텐마감이 안되어있어서 검지손가락...,1


In [ ]:
test.head()

,id,reviews
0,0,채소가 약간 시들어 있어요
1,1,발톱 두껍고 단단한 분들 써도 소용없어요 이 테이프 물렁거리고 힘이없어서 들어 올리...
2,2,부들부들 좋네요 입어보고 시원하면 또 살게요
3,3,이런 1. 8 골드 주라니깐 파란개 오네 회사전화걸어도 받지도 않고 머하자는거임?
4,4,검수도 없이 보내구 불량 배송비 5000원 청구하네요 완전별로 별하나도 아까워요


In [ ]:
train.shape, test.shape

((25000, 3), (25000, 2))

## spell_checker

In [ ]:
train1 = train.iloc[:10000,:]

In [ ]:
def spellchecker(x):
    x = re.sub('\W', ' ', x)
    sc = spell_checker.check(x).checked
    return sc
train1['reviews_ck'] = train1['reviews'].apply(spellchecker)
train1.head()

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """


,id,reviews,target,reviews_ck
0,0,조아요 처음구입 싸게햇어요,2,좋아요 처음 구입 싸게 했어요
1,1,생각보다 잘 안돼요 매지 바른지 하루밖에 안됐는데ㅠㅠ 25천원가량 주고 사기 너무 ...,1,생각보다 잘 안돼요 매지 바른지 하루밖에 안됐는데ㅠㅠ 25천 원가량 주고 사기 너무...
2,2,디자인은괜찮은데 상품이 금이가서 교환했는데 두번째받은상품도 까져있고 안쪽에 금이가져...,2,디자인은 괜찮은데 상품이 금이 가서 교환했는데 두 번째 받은 상품도 까져있고 안쪽에...
3,3,기전에 이 제품말고 이마트 트레이더스에서만 팔던 프리미엄 제품을 사용했었습니다. 샘...,2,기전에 이 제품 말고 이마트 트레이더스에서만 팔던 프리미엄 제품을 사용했었습니다 ...
4,4,튼튼하고 손목을 잘 받쳐주네요~,5,튼튼하고 손목을 잘 받쳐주네요


In [ ]:
train2 = train.iloc[10000:20000,:]

In [ ]:
def spellchecker(x):
    x = re.sub('\W', ' ', x)
    sc = spell_checker.check(x).checked
    return sc
train2['reviews_ck'] = train2['reviews'].apply(spellchecker)
train2.head()

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """


,id,reviews,target,reviews_ck
10000,10000,완전별로요,2,완전별로요
10001,10001,요거 쓰고나면 다른 제품은 만족스럽지가 않아요ㅎㅎ 그만큼 깨끗하고 락스냄새도 안나고...,5,요거 쓰고 나면 다른 제품은 만족스럽지가 않아요ㅎㅎ 그만큼 깨끗하고 락스 냄새도 안...
10002,10002,재구매 매번 구매하고 있는제 빠른 배송과 착한가격 너무나도 잘 이용하고 있습니다. ^^,5,재구매 매번 구매하고 있는 제 빠른 배송과 착한 가격 너무나도 잘 이용하고 있습니다...
10003,10003,굳굳 좋아요! 잘쓰는 중,2,굳굳 좋아요 잘 쓰는 중
10004,10004,엄청 맛있진 않고 쏘쏘 인거 같아요,2,엄청 맛있진 않고 쏘소인 거 같아요


In [ ]:
train3 = train.iloc[20000:,:]

In [ ]:
def spellchecker(x):
    x = re.sub('\W', ' ', x)
    sc = spell_checker.check(x).checked
    return sc
train3['reviews_ck'] = train3['reviews'].apply(spellchecker)
train3.head()

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """


,id,reviews,target,reviews_ck
20000,20000,신발정리하기좋킨한데부실해요,2,신발정리하기 좋킨 한데 부실해요
20001,20001,너무싸구려틱해요ㆍ,2,너무싸구려틱해요ㆍ
20002,20002,너무약해서 옷을걸을때마다 흔들거려쓰러질거같아요 디자인은 괜찮아요,2,너무 약해서 옷을 걸을 때마다 흔들거려 쓰러질 거 같아요 디자인은 괜찮아요
20003,20003,배송이 엉망 확인에는 배달완료로 뜨는데 배송은 안되고 기사님 전화는 안받고 기다려라...,1,배송이 엉망 확인에는 배달 완료로 뜨는데 배송은 안되고 기사님 전화는 안 받고 기다...
20004,20004,재구매 사진첨부가 안되네요. 저번에도 여기서 구입했어요,1,재구매 사진 첨부가 안 되네요 저번에도 여기서 구입했어요


In [ ]:
train_ck = pd.concat([train1, train2,train3], axis=0)
print(train_ck.head())
print(train_ck.tail())

   id                                            reviews  target  \
0   0                                     조아요 처음구입 싸게햇어요       2   
1   1  생각보다 잘 안돼요 매지 바른지 하루밖에 안됐는데ㅠㅠ 25천원가량 주고 사기 너무 ...       1   
2   2  디자인은괜찮은데 상품이 금이가서 교환했는데 두번째받은상품도 까져있고 안쪽에 금이가져...       2   
3   3  기전에 이 제품말고 이마트 트레이더스에서만 팔던 프리미엄 제품을 사용했었습니다. 샘...       2   
4   4                                  튼튼하고 손목을 잘 받쳐주네요~       5   

                                          reviews_ck  
0                                   좋아요 처음 구입 싸게 했어요  
1  생각보다 잘 안돼요 매지 바른지 하루밖에 안됐는데ㅠㅠ 25천 원가량 주고 사기 너무...  
2  디자인은 괜찮은데 상품이 금이 가서 교환했는데 두 번째 받은 상품도 까져있고 안쪽에...  
3  기전에 이 제품 말고 이마트 트레이더스에서만 팔던 프리미엄 제품을 사용했었습니다  ...  
4                                  튼튼하고 손목을 잘 받쳐주네요   
          id                                      reviews  target  \
24995  24995          일단향이너무너무좋은데지속력만좋다면좋겠지만..워낙저렴해그래도좋아여       5   
24996  24996  동일한 업체에서 발송하고 한 상자로 배송되었는데 택배비는 이중 처리되었습니다.       2   
24997  24997                      향이 좋아요!! 다른 향도 시켜보고 

In [ ]:
train_ck.to_csv('train_ck.csv', index=False)

In [3]:
train_ck = pd.read_csv('/content/train_ck.csv')
train_ck.head()

,id,reviews,target,reviews_ck
0,0,조아요 처음구입 싸게햇어요,2,좋아요 처음 구입 싸게 했어요
1,1,생각보다 잘 안돼요 매지 바른지 하루밖에 안됐는데ㅠㅠ 25천원가량 주고 사기 너무 ...,1,생각보다 잘 안돼요 매지 바른지 하루밖에 안됐는데ㅠㅠ 25천 원가량 주고 사기 너무...
2,2,디자인은괜찮은데 상품이 금이가서 교환했는데 두번째받은상품도 까져있고 안쪽에 금이가져...,2,디자인은 괜찮은데 상품이 금이 가서 교환했는데 두 번째 받은 상품도 까져있고 안쪽에...
3,3,기전에 이 제품말고 이마트 트레이더스에서만 팔던 프리미엄 제품을 사용했었습니다. 샘...,2,기전에 이 제품 말고 이마트 트레이더스에서만 팔던 프리미엄 제품을 사용했었습니다 ...
4,4,튼튼하고 손목을 잘 받쳐주네요~,5,튼튼하고 손목을 잘 받쳐주네요


In [7]:
train_ck.tail()

,id,reviews,target,reviews_ck
24995,24995,일단향이너무너무좋은데지속력만좋다면좋겠지만..워낙저렴해그래도좋아여,5,일단 향이 너무너무 좋은데 지 속력만 좋다면 좋겠지만 워낙 저렴해 그래도 좋아요
24996,24996,동일한 업체에서 발송하고 한 상자로 배송되었는데 택배비는 이중 처리되었습니다.,2,동일한 업체에서 발송하고 한 상자로 배송되었는데 택배비는 이중 처리되었습니다
24997,24997,향이 좋아요!! 다른 향도 시켜보고 싶어요,4,향이 좋아요 다른 향도 시켜보고 싶어요
24998,24998,펼쳐보니 끝부분쯤에 50cm정도 찟어져 있습니다 날짜관계로 구냥 사용합니다,2,펼쳐보니 끝부분쯤에 50cm 정도 찢어져 있습니다 날짜 관계로 구냥 사용합니다
24999,24999,호호애미 제품은 늘 믿고 써요,5,호호 어미 제품은 늘 믿고 써요


In [8]:
test1 = test.iloc[:10000,:]
def spellchecker(x):
    x = re.sub('\W', ' ', x)
    sc = spell_checker.check(x).checked
    return sc
test1['reviews_ck'] = test1['reviews'].apply(spellchecker)
test1.head()

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


,id,reviews,reviews_ck
0,0,채소가 약간 시들어 있어요,채소가 약간 시들어 있어요
1,1,발톱 두껍고 단단한 분들 써도 소용없어요 이 테이프 물렁거리고 힘이없어서 들어 올리...,발톱 두껍고 단단한 분들 써도 소용없어요 이 테이프 물렁거리고 힘이 없어서 들어 올...
2,2,부들부들 좋네요 입어보고 시원하면 또 살게요,부들부들 좋네요 입어보고 시원하면 또 살게요
3,3,이런 1. 8 골드 주라니깐 파란개 오네 회사전화걸어도 받지도 않고 머하자는거임?,이런 1 8 골드 주라니깐 파란 개 오네 회사 전화 걸어도 받지도 않고 머 하자는 거임
4,4,검수도 없이 보내구 불량 배송비 5000원 청구하네요 완전별로 별하나도 아까워요,검수도 없이 보내고 불량 배송비 5000원 청구하네요 완전별로 별 하나도 아까워요


In [9]:
test2 = test.iloc[10000:20000,:]
def spellchecker(x):
    x = re.sub('\W', ' ', x)
    sc = spell_checker.check(x).checked
    return sc
test2['reviews_ck'] = test2['reviews'].apply(spellchecker)
test2.head()

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


,id,reviews,reviews_ck
10000,10000,편안하게 집에서 입고 수유하기 좋아요. 면으로 자연스러워서 입기 좋네요,편안하게 집에서 입고 수유하기 좋아요 면으로 자연스러워서 입기 좋네요
10001,10001,이거 입고 운동도 하나요? 수면속옷으로 구입했지만 한번 입어보니 절대적으로 수면용이...,이거 입고 운동도 하나요 수면 속옷으로 구입했지만 한번 입어보니 절대적으로 수면용...
10002,10002,완전 마음에들어요! 깔끔해보이고 수납력이좋습니다,완전 마음에 들어요 깔끔해 보이고 수납력이좋습니다
10003,10003,불빛스위치가 너무 허술함,불빛 스위치가 너무 허술함
10004,10004,아주 좋습니다 향도 좋구요,아주 좋습니다 향도 좋고요


In [10]:
test3 = test.iloc[20000:,:]
def spellchecker(x):
    x = re.sub('\W', ' ', x)
    sc = spell_checker.check(x).checked
    return sc
test3['reviews_ck'] = test3['reviews'].apply(spellchecker)
test3.head()

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


,id,reviews,reviews_ck
20000,20000,병머리가 불량이라 뚝하고 부러져서 액이사방으로 다튀었네요 조심들하세요 확인후 장착하세요,병머리가 불량이라 뚝하고 부러져서 액이 사방으로 다 튀었네요 조심들 하세요 확인 후...
20001,20001,쏘쏘...느리고 안터지고,쏘 쏘 느리고 안 터지고
20002,20002,고객이 회사에 어떤 연락도 취할 수 없네요. 반품신청 했는데 네이버페이를 통해서 연...,고객이 회사에 어떤 연락도 취할 수 없네요 반품 신청했는데 네이버 페이를 통해서 ...
20003,20003,재구매 무난하게 잘 사용하고 있어요!,재구매 무난하게 잘 사용하고 있어요
20004,20004,톡톡상담 답변 빨리주셔서 감사합니다!,톡톡 상담 답변 빨리 주셔서 감사합니다


In [11]:
test_ck = pd.concat([test1, test2, test3], axis=0)
print(test_ck.head())
print(test_ck.tail())

   id                                            reviews  \
0   0                                     채소가 약간 시들어 있어요   
1   1  발톱 두껍고 단단한 분들 써도 소용없어요 이 테이프 물렁거리고 힘이없어서 들어 올리...   
2   2                           부들부들 좋네요 입어보고 시원하면 또 살게요   
3   3      이런 1. 8 골드 주라니깐 파란개 오네 회사전화걸어도 받지도 않고 머하자는거임?   
4   4       검수도 없이 보내구 불량 배송비 5000원 청구하네요 완전별로 별하나도 아까워요   

                                          reviews_ck  
0                                     채소가 약간 시들어 있어요  
1  발톱 두껍고 단단한 분들 써도 소용없어요 이 테이프 물렁거리고 힘이 없어서 들어 올...  
2                           부들부들 좋네요 입어보고 시원하면 또 살게요  
3  이런 1  8 골드 주라니깐 파란 개 오네 회사 전화 걸어도 받지도 않고 머 하자는 거임  
4      검수도 없이 보내고 불량 배송비 5000원 청구하네요 완전별로 별 하나도 아까워요  
          id                                            reviews  \
24995  24995                                       사용해보니 좋아요~^^   
24996  24996                 저렴한가격에. 질좋고. 핏좋고. 너무. 이쁘게. 입고다녀요..   
24997  24997  세트상품이라고 써있어서 그런줄 알고 구매했더니 단품이었네요 낚인 느낌도 들고 그러네...   
24998  24998                             

In [12]:
test_ck.to_csv('test_ck.csv', index=False)

In [2]:
test_ck = pd.read_csv('/content/test_ck.csv')

## token

In [4]:
okt = Okt()
train_tk = train_ck.copy()
def token(x):
    re = []
    for word in okt.pos(x): 
        if word[1] in ['Noun', 'Verb', 'Adjective']: #명사, 동사, 형용사
            re.append(word[0])
    return ' '.join(re)
train_tk['reviews_tk'] = train_ck['reviews_ck'].apply(token)

test_tk = test_ck.copy()
test_tk['reviews_tk'] = test_ck['reviews_ck'].apply(token)

In [9]:
train_tk.tail()

,id,reviews,target,reviews_ck,reviews_tk
24995,24995,일단향이너무너무좋은데지속력만좋다면좋겠지만..워낙저렴해그래도좋아여,5,일단 향이 너무너무 좋은데 지 속력만 좋다면 좋겠지만 워낙 저렴해 그래도 좋아요,일단 향 좋은데 지 속력 좋다면 좋겠지만 워낙 저렴해 좋아요
24996,24996,동일한 업체에서 발송하고 한 상자로 배송되었는데 택배비는 이중 처리되었습니다.,2,동일한 업체에서 발송하고 한 상자로 배송되었는데 택배비는 이중 처리되었습니다,동일한 업체 발송 한 상자 배송 되었는데 택배 비는 이중 처리 되었습니다
24997,24997,향이 좋아요!! 다른 향도 시켜보고 싶어요,4,향이 좋아요 다른 향도 시켜보고 싶어요,향 좋아요 다른 향 시켜 보고 싶어요
24998,24998,펼쳐보니 끝부분쯤에 50cm정도 찟어져 있습니다 날짜관계로 구냥 사용합니다,2,펼쳐보니 끝부분쯤에 50cm 정도 찢어져 있습니다 날짜 관계로 구냥 사용합니다,펼쳐 보니 끝 부분 정도 찢어져 있습니다 날짜 관계 구 사용 합니다
24999,24999,호호애미 제품은 늘 믿고 써요,5,호호 어미 제품은 늘 믿고 써요,호호 어미 제품 늘 믿고 써요


## sklearn(countervector)(train1)

In [5]:
ctv = sklearn.feature_extraction.text.CountVectorizer()
x_train1 = ctv.fit_transform(train_tk['reviews_tk'])
x_test1 = ctv.transform(test_tk['reviews_tk'])
y_train1 = train_tk['target']
x_train1.shape, y_train1.shape, x_test1.shape

((25000, 25442), (25000,), (25000, 25442))

## tf(labeling+pading)(train3)

In [26]:
okt = Okt()
train_tk = train_ck.copy()
def token(x):
    re = []
    for word in okt.pos(x): 
        if word[1] in ['Noun', 'Verb', 'Adjective']: #명사, 동사, 형용사
            re.append(word[0])
    return re
train_tk['reviews_tk'] = train_ck['reviews_ck'].apply(token)

test_tk = test_ck.copy()
test_tk['reviews_tk'] = test_ck['reviews_ck'].apply(token)

In [32]:
tokenizer = tf.keras.preprocessing.text.Tokenizer()
tokenizer.fit_on_texts(train_tk['reviews_tk'])
train_tk['reviews_lb'] = tokenizer.texts_to_sequences(train_tk['reviews_tk'])
test_tk['reviews_lb'] = tokenizer.texts_to_sequences(test_tk['reviews_tk'])

In [33]:
train_tk.head(2)

,id,reviews,target,reviews_ck,reviews_tk,reviews_lb
0,0,조아요 처음구입 싸게햇어요,2,좋아요 처음 구입 싸게 했어요,"[좋아요, 처음, 구입, 싸게, 했어요]","[4, 76, 58, 382, 36]"
1,1,생각보다 잘 안돼요 매지 바른지 하루밖에 안됐는데ㅠㅠ 25천원가량 주고 사기 너무 ...,1,생각보다 잘 안돼요 매지 바른지 하루밖에 안됐는데ㅠㅠ 25천 원가량 주고 사기 너무...,"[생각, 잘, 돼요, 매지, 바른지, 하루, 됐는데, 가량, 주고, 사기, 아깝네요]","[13, 1, 230, 8950, 12753, 208, 918, 2994, 293,..."


In [37]:
tf_index = tokenizer.word_index
tf_index['<PAD>'] = 0
word_size = len(tf_index) + 1
word_size

26628

In [ ]:
tf.keras.preprocessing.sequence.pad_sequences()

# modeling

## sk

In [6]:
x_ttrain, x_ttest, y_ttrain, y_ttest = train_test_split(x_train1, y_train1, stratify=y_train1)

In [16]:
knc = KNeighborsClassifier()
svc = SVC(probability=True)
rfc = RandomForestClassifier()
xgb = XGBRFClassifier()
lgbm = LGBMClassifier()
cat = CatBoostClassifier(verbose=0)

In [8]:
models = [knc, svc, rfc, xgb, cat]
for i in models:
    model = i.fit(x_ttrain, y_ttrain)
    pre = model.predict(x_ttest)
    sc = round(model.score(x_ttrain, y_ttrain), 4)
    rasc = round(accuracy_score(y_ttest, pre), 4)
    print(f'{model.__class__.__name__} == sc : {sc}, rasc : {rasc}')

KNeighborsClassifier == sc : 0.6597, rasc : 0.4838
SVC == sc : 0.829, rasc : 0.6307
RandomForestClassifier == sc : 0.9935, rasc : 0.6054
XGBRFClassifier == sc : 0.4587, rasc : 0.4611
CatBoostClassifier == sc : 0.6599, rasc : 0.6253


In [10]:
params={}
gsv = GridSearchCV(svc, params, scoring='accuracy', cv=5)
model = gsv.fit(x_train1, y_train1)
print(model.best_score_)
print(model.best_params_)

0.6322000000000001
{}


In [14]:
params={'n_estimators' : [50,40,30], 'max_depth' : [10,9,12]}
gsv = GridSearchCV(rfc, params, scoring='accuracy', cv=5)
model = gsv.fit(x_train1, y_train1)
print(model.best_score_)
print(model.best_params_)

0.45244
{'max_depth': 12, 'n_estimators': 30}


In [17]:
voc = VotingClassifier([('svc',svc),('cat',cat)],voting='soft')
model = voc.fit(x_train1, y_train1)
pre = model.predict(x_test1)

In [18]:
pre

array([5, 2, 5, ..., 2, 5, 1])

In [19]:
sub = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/데이콘/쇼핑몰 리뷰 평점 분류 경진대회/sample_submission.csv')

In [21]:
sub['target']=pre

In [23]:
sub.to_csv('submission.csv', index=False)

## tf